In [2]:
df = DataSource("bar1d_CN_FUND").read(start_date="2020-06-01", end_date="2020-06-30")

,instrument,date,open,high,low,close,volume,amount,adjust_factor,turn


In [77]:
# set(m11.data_1.read_df()['instrument'].tolist())
m22.data.read_df()

,instrument,date,open,high,low,close,volume,amount,adjust_factor
0,510050.HOF,2018-01-02,3.605200,3.661900,3.602700,3.659400,477697012.0,1.383492e+09,1.258824
1,510230.HOF,2018-01-02,6.320000,6.431000,6.320000,6.411000,1273300.0,8.158987e+06,1.000000
2,511010.HOF,2018-01-02,110.933899,110.933899,110.775002,110.846901,248100.0,2.717178e+07,1.012116
3,511260.HOF,2018-01-02,98.119003,98.199997,98.001999,98.199997,107040.0,1.051118e+07,1.000000
4,512500.HOF,2018-01-02,3.042000,3.100000,3.042000,3.086000,180547.0,5.551740e+05,1.000000
...,...,...,...,...,...,...,...,...,...
18191,159864.ZOF,2021-12-31,1.011000,1.042000,1.011000,1.032000,105154340.0,1.088652e+08,1.000000
18192,159865.ZOF,2021-12-31,0.875000,0.901000,0.873000,0.899000,104388313.0,9.286924e+07,1.000000
18193,159881.ZOF,2021-12-31,1.208000,1.220000,1.207000,1.216000,23231600.0,2.820375e+07,1.000000
18194,159806.ZOF,2021-12-31,3.024000,3.054000,3.006000,3.025000,16162204.0,4.893974e+07,1.000000


In [96]:
# 本代码由可视化策略环境自动生成 2023年8月18日 09:00
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。
 
# 显式导入 BigQuant 相关 SDK 模块
from bigdatasource.api import DataSource
from biglearning.api import M
from biglearning.api import tools as T
from biglearning.module2.common.data import Outputs
 
import pandas as pd
import numpy as np
import math
import warnings
import datetime
 
from zipline.finance.commission import PerOrder
from zipline.api import get_open_orders
from zipline.api import symbol
 
from bigtrader.sdk import *
from bigtrader.utils.my_collections import NumPyDeque
from bigtrader.constant import OrderType
from bigtrader.constant import Direction

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m11_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    instruments = input_1.read()['instruments']
    start_date = input_1.read()['start_date']
    end_date = input_1.read()['end_date']
    df = DataSource('bar1d_CN_FUND').read(instruments=instruments,start_date=start_date,end_date=end_date)
    df = df.drop(columns=['turn'])
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m11_post_run_bigquant_run(outputs):
    return outputs

# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m24_run_bigquant_run(input_1, input_2, input_3):
    # 示例代码如下。在这里编写您的代码
    instruments = input_1.read()['instruments']
    start_date = input_1.read()['start_date']
    end_date = input_1.read()['end_date']
    df = DataSource('bar1d_CN_FUND').read(instruments=instruments,start_date=start_date,end_date=end_date)
    df = df.drop(columns=['turn'])
    
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m24_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):
    # 加载预测数据
    context.ranker_prediction = context.options['data'].read_df()

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.00015, sell_cost=0.00015, min_cost=0))
    # 预测数据，通过options传入进来，使用 read_df 函数，加载到内存 (DataFrame)
    # 设置买入的股票数量，这里买入预测股票列表排名靠前的5只
    stock_count = 5
    # 每只的股票的权重，如下的权重分配会使得靠前的股票分配多一点的资金，[0.339160, 0.213986, 0.169580, ..]
    context.stock_weights = T.norm([1 / math.log(i + 2) for i in range(0, stock_count)])
    # 设置每只股票占用的最大资金比例
    context.max_cash_per_instrument = 0.2
    context.options['hold_days'] = 2

# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    # 按日期过滤得到今日的预测数据
    ranker_prediction = context.ranker_prediction[
        context.ranker_prediction.date == data.current_dt.strftime('%Y-%m-%d')]
    today = data.current_dt.strftime('%Y-%m-%d')
    # 1. 资金分配
    # 平均持仓时间是hold_days，每日都将买入股票，每日预期使用 1/hold_days 的资金
    # 实际操作中，会存在一定的买入误差，所以在前hold_days天，等量使用资金；之后，尽量使用剩余资金（这里设置最多用等量的1.5倍）
    is_staging = context.trading_day_index < context.options['hold_days'] # 是否在建仓期间（前 hold_days 天）
    cash_avg = context.portfolio.portfolio_value / context.options['hold_days']
    cash_for_buy = min(context.portfolio.cash, (1 if is_staging else 1.5) * cash_avg)
    cash_for_sell = cash_avg - (context.portfolio.cash - cash_for_buy)
    positions = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.portfolio.positions.items()}

    # 2. 生成卖出订单：hold_days天之后才开始卖出；对持仓的股票，按机器学习算法预测的排序末位淘汰
    if not is_staging and cash_for_sell > 0:
        equities = {e.symbol: e for e, p in context.portfolio.positions.items()}
        instruments = list(reversed(list(ranker_prediction.instrument[ranker_prediction.instrument.apply(
                lambda x: x in equities)])))

        for instrument in instruments:
            context.order_target(context.symbol(instrument), 0)
            cash_for_sell -= positions[instrument]
            if cash_for_sell <= 0:
                break

    # 3. 生成买入订单：按机器学习算法预测的排序，买入前面的stock_count只股票
    buy_cash_weights = context.stock_weights
    buy_instruments = list(ranker_prediction.instrument[:len(buy_cash_weights)])
    max_cash_per_instrument = context.portfolio.portfolio_value * context.max_cash_per_instrument
    for i, instrument in enumerate(buy_instruments):
        cash = cash_for_buy * buy_cash_weights[i]
        if cash > max_cash_per_instrument - positions.get(instrument, 0):
            # 确保股票持仓量不会超过每次股票最大的占用资金量
            cash = max_cash_per_instrument - positions.get(instrument, 0)
        if cash > 0:
            context.order_value(context.symbol(instrument), cash)

# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    pass


m1 = M.instruments.v2(
    start_date='2018-01-01',
    end_date='2022-01-01',
    market='CN_FUND',
    instrument_list="""588360.HOF
561350.HOF
561310.HOF
561300.HOF
518800.HOF
517500.HOF
517110.HOF
516960.HOF
516620.HOF
516220.HOF
516110.HOF
516010.HOF
515880.HOF
515230.HOF
515220.HOF
515210.HOF
513100.HOF
513020.HOF
512880.HOF
512760.HOF
512720.HOF
512660.HOF
512290.HOF
511260.HOF
511010.HOF
510760.HOF
510230.HOF
159996.ZOF
159889.ZOF
159881.ZOF
159865.ZOF
159864.ZOF
159861.ZOF
159828.ZOF
159806.ZOF
159761.ZOF
159745.ZOF
159712.ZOF
159619.ZOF
159601.ZOF
510050.HOF
588000.HOF
159967.ZOF
159966.ZOF
512500.HOF
515170.HOF
159985.ZOF
159845.ZOF
561910.HOF
""",
    max_count=0
)

m11 = M.cached.v3(
    input_1=m1.data,
    run=m11_run_bigquant_run,
    post_run=m11_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m5 = M.auto_labeler_on_datasource.v1(
    input_data=m11.data_1,
    label_expr="""# #号开始的表示注释
# 0. 每行一个，顺序执行，从第二个开始，可以使用label字段
# 1. 可用数据字段见 https://bigquant.com/docs/develop/datasource/deprecated/history_data.html
# 2. 可用操作符和函数见 `表达式引擎 <https://bigquant.com/docs/develop/bigexpr/usage.html>`_

# 计算收益：5日收盘价(作为卖出价格)除以明日开盘价(作为买入价格)
shift(close, -2) / shift(open, -1)

# 极值处理：用1%和99%分位的值做clip
clip(label, all_quantile(label, 0.01), all_quantile(label, 0.99))

# 将分数映射到分类，这里使用20个分类
all_wbins(label, 20)

""",
    drop_na_label=True,
    cast_label_int=True,
    date_col='date',
    instrument_col='instrument',
    user_functions={}
)

m13 = M.general_feature_extractor.v7(
    instruments=m1.data,
    start_date='2018-01-01',
    end_date='2022-01-01',
    before_start_days=90
)

m23 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
# close/shift(close, 2)-1 
# std(close, 2)
# decay_linear(close, 2)
# ts_max(close, 2)
# ts_min(close, 2)
close_0
low_0
volume_0
return_0
high_0
open_0
pb_lf_0
ta_atr_14_0
ta_bbands_lowerband_14_0
ta_wma_5_0

((((sum(close_0,7)/7)-close_0))+((correlation((high_0+low_0+close_0+open_0)/4,delay(close_0,5),230))))
((((-1*ts_min(low_0,5))+delay(ts_min(low_0,5),5))*rank(((sum(return_0,240)-sum(return_0,20))/220)))*ts_rank(volume_0,5))
mean(max(max((high_0-low_0),abs(delay(close_0,1)-high_0)),abs(delay(close_0,1)-low_0)),6)
((((-1*ts_min(low_0,5))+delay(ts_min(low_0,5),5))*rank(((sum((close_0/shift(close_0,1)-1),240)-sum((close_0/shift(close_0,1)-1),20))/220)))*ts_rank(volume_0,5))
mean(max(max((high_0-low_0),abs(delay(close_0,1)-high_0)),abs(delay(close_0,1)-low_0)),12)
sum(where(close_0-delay(close_0,1)<0,abs(close_0-delay(close_0,1)),0),12)
((-1*rank((delta(close_0,7)*(1-rank(decay_linear(volume_0/mean(volume_0,20),9))))))*(1+rank(sum((close_0/shift(close_0,1)-1),250))))
(close_0+high_0+low_0)/3"""
)

m22 = M.derived_feature_extractor.v3(
    input_data=m13.data,
    features=m23.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False,
    user_functions={}
)

m12 = M.join.v3(
    data1=m5.data,
    data2=m22.data,
    on='date,instrument',
    how='inner',
    sort=True
)

m19 = M.dropnan.v1(
    input_data=m12.data
)

m21 = M.stock_ranker_train.v5(
    training_ds=m19.data,
    features=m23.data,
    test_ds=m19.data,
    learning_algorithm='排序',
    number_of_leaves=20,
    minimum_docs_per_leaf=120,
    number_of_trees=30,
    learning_rate=0.1,
    max_bins=1023,
    feature_fraction=1,
    m_lazy_run=False
)

m24 = M.cached.v3(
    input_1=m23.data,
    run=m24_run_bigquant_run,
    post_run=m24_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m25 = M.derived_feature_extractor.v3(
    input_data=m24.data_1,
    date_col='date',
    instrument_col='instrument',
    drop_na=True,
    remove_extra_columns=False,
    user_functions={}
)

m26 = M.dropnan.v1(
    input_data=m25.data
)

m20 = M.stock_ranker_predict.v5(
    model=m21.model,
    data=m26.data,
    m_lazy_run=False
)

m10 = M.instruments.v2(
    start_date='2022-01-02',
    end_date='2022-05-01',
    market='CN_FUND',
    instrument_list="""588360.HOF
561350.HOF
561310.HOF
561300.HOF
518800.HOF
517500.HOF
517110.HOF
516960.HOF
516620.HOF
516220.HOF
516110.HOF
516010.HOF
515880.HOF
515230.HOF
515220.HOF
515210.HOF
513100.HOF
513020.HOF
512880.HOF
512760.HOF
512720.HOF
512660.HOF
512290.HOF
511260.HOF
511010.HOF
510760.HOF
510230.HOF
159996.ZOF
159889.ZOF
159881.ZOF
159865.ZOF
159864.ZOF
159861.ZOF
159828.ZOF
159806.ZOF
159761.ZOF
159745.ZOF
159712.ZOF
159619.ZOF
159601.ZOF
510050.HOF
588000.HOF
159967.ZOF
159966.ZOF
512500.HOF
515170.HOF
159985.ZOF
159845.ZOF
561910.HOF
""",
    max_count=0
)

m3 = M.trade.v4(
    instruments=m10.data,
    options_data=m20.predictions,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    volume_limit=1,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=10000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark='000300.HIX'
)

m2 = M.strategy_ret_risk_analysis.v2(
    input_1=m3.raw_perf,
    analysis_flag='relative',
    benchmark_index='000300.HIX',
    terms='daily'
)

m4 = M.strategy_average_position_analysis.v1(
    raw_perf=m3.raw_perf
)

m7 = M.strategy_income_distribution.v1(
    raw_perf=m3.raw_perf
)

m8 = M.strategy_interval_return.v1(
    raw_perf=m3.raw_perf,
    keep_n_yearly=6,
    keep_n_quarterly=8,
    keep_n_monthly=8
)

m9 = M.strategy_evaluate_risk_overview.v1(
    raw_perf=m3.raw_perf
)

[2022-05-08 22:57:49.213219] INFO: moduleinvoker: instruments.v2 开始运行..

[2022-05-08 22:57:49.222489] INFO: moduleinvoker: 命中缓存

[2022-05-08 22:57:49.225181] INFO: moduleinvoker: instruments.v2 运行完成[0.011998s].

[2022-05-08 22:57:49.238464] INFO: moduleinvoker: general_feature_extractor.v7 开始运行..

[2022-05-08 22:57:49.250141] ERROR: moduleinvoker: module name: general_feature_extractor, module version: v7, trackeback: TypeError: __init__() takes at least 3 positional arguments (2 given)\n

TypeError: __init__() takes at least 3 positional arguments (2 given)